In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from spafe.features.gfcc import gfcc
import librosa
from sklearn import mixture
from tqdm import tqdm
# import cPickle
# !pip install cPickle
import pickle
import librosa
import numpy as np
from librosa.core.spectrum import _spectrogram
import os
import soundfile as sf
from tqdm import tqdm
from audiomentations import AddBackgroundNoise, PolarityInversion
from audiomentations import Compose, AddGaussianNoise, PitchShift, HighPassFilter,\
    AddGaussianSNR,AddShortNoises,ApplyImpulseResponse,BandPassFilter,Gain,TanhDistortion,\
    HighShelfFilter,HighPassFilter,LowPassFilter,LowShelfFilter,Limiter,LoudnessNormalization
import random
from sklearn import mixture
import pickle
import pyrubberband
import random

In [ ]:
def extract_features(path):
    x,sr = librosa.load(path)
    x_pre_emp = librosa.effects.preemphasis(x)
    x_mfcc = librosa.feature.mfcc(y=x_pre_emp,sr=sr,n_mfcc=13)

    delta_mfcc = librosa.feature.delta(x_mfcc)
    double_delta_mfcc = librosa.feature.delta(delta_mfcc)
    combined = np.vstack((x_mfcc,delta_mfcc,double_delta_mfcc))
    return combined.T

In [ ]:
model_path = '../ML_project/Models2'
models = [pickle.load(open(model_path+'/'+fname,'rb')) for fname in os.listdir(model_path)]
speakers   = [fname.split('.')[0] for fname in os.listdir(model_path)]
folder_path = ""  # your path goes here (if you give folder)
y_pred = []
y_pred_dict = {}
path1 = 'path_to_folder'
testing = os.listdir(path1)
# list_of_paths = []   # list of paths given at testing time
for i in tqdm(testing):
    path = i
    features = extract_features(path1+'/'+path)
    log_likelihood = np.zeros(len(models)) 
    for j in range(len(models)):
        gmm = models[j]         #checking with each model one by one
        scores = np.array(gmm.score(features))
        log_likelihood[j] = scores.sum() 

    winner = np.argmax(log_likelihood) 
    y_pred.append(speakers[winner]) 
    y_pred_dict[i] = speakers[winner]
y_pred_dict

